# Iteracyjne metody rozwiązywania równań liniowych

### Czytanka
* Kincaid, Cheney, rozdz. 8.2, str. 319 (bardzo przystępnie napisane)
* Normy wektorów i macierzy:
    * wektorowa: https://en.wikipedia.org/wiki/Norm_(mathematics)
    * macierzowa: https://en.wikipedia.org/wiki/Matrix_norm
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method
* Metoda SOR: https://en.wikipedia.org/wiki/Successive_over-relaxation
* Metoda Gaussa-Seidela: https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method
* (ale wystarczy K&C)

### Troszkę teorii

Chcemy rozwiązać układ równań liniowych postaci $A\mathbf {x} =\mathbf {b} $, gdzie:

$$
A={\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}},\qquad \mathbf {x} ={\begin{bmatrix}x_{1}\\x_{2}\\\vdots \\x_{n}\end{bmatrix}},\qquad \mathbf {b} ={\begin{bmatrix}b_{1}\\b_{2}\\\vdots \\b_{n}\end{bmatrix}}.
$$

Mimo, że dobrze znamy dokładne metody rozwiązania takiego równania, częstokroć w praktyce nie możemy ich zastosować -- przede wszystkim ze względu na rozmiar problemu. Rozwiązanie? Zastosować metody iteracyjne, które, choć nie dadzą nam dokładnego wyniku, pozwolą nam w rozsądnym czasie uzyskać dobrą aproksymację. (Zresztą, dokładne metody też nie dają dokładnych rezultatów z powodu błędów arytmetyki zmiennoprzecinkowej).

_Suchy żarcik dnia: John ma problem. John myśli: "Wiem, użyję arytmetyki zmiennoprzecinkowej." Teraz John ma 1.999999997 problemu. (Zasłyszane w pracy)._


#### Metoda Jacobiego

Metody Jacobiego możemy użyć pod warunkiem, że macierz jest przekątniowo dominująca, tj. mamy $ |a_{ii}|\geq \sum _{j\neq i}|a_{ij}|\quad {\text{dla każdego }}i. $

Pomysł polega na rozkładzie macierzy A na **sumę** dwóch macierzy:
$$
A=D+R\qquad {\text{gdzie}}\qquad D={\begin{bmatrix}a_{11}&0&\cdots &0\\0&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &a_{nn}\end{bmatrix}}{\text{ oraz }}R={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\a_{21}&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &0\end{bmatrix}}.
$$

Następnie krok iteracji wygląda następująco:
$$ \mathbf {x} ^{(k+1)}=D^{-1}(\mathbf {b} -R\mathbf {x} ^{(k)}), $$

I element po elemencie:
$$ x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j\neq i}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\ldots ,n. $$

Zwróćmy uwagę, że cały trick polega na tym, że macierz $D$ bardzo łatwo odwrócić.

### Zadanie 1.
Zaimplementować metodę Jacobiego i przetestować jej działanie na paru układach równań. Porównać z metodą eliminacji Gaussa użytą do tych samych równań.

In [36]:
def check_method(A: np.matrix, b: np.matrix, method):
    xt = np.linalg.solve(A, b)
    for i in range(1,1000):
        x = method(A, b, i)
        print(x)
        if(np.allclose(xt.transpose(), x) == True):
            print("Accurate after", i, "iterations")
            break

In [23]:
import numpy as np

def jacobi_solve(A: np.matrix, b: np.matrix, iterations = 1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum = sum + A[i, j] * x[j]
            x1[i] = (1 / A[i, i]) * (b[i] - sum)
        x = x1
        k = k + 1
    return x

In [25]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, jacobi_solve)

[5.5        1.85714286]
[ 4.57142857 -2.07142857]
[ 6.53571429 -1.40816327]
[ 6.20408163 -2.81122449]
[ 6.90561224 -2.57434402]
[ 6.78717201 -3.07543732]
[ 7.03771866 -2.99083715]
[ 6.99541858 -3.16979904]
[ 7.08489952 -3.1395847 ]
[ 7.06979235 -3.20349966]
[ 7.10174983 -3.19270882]
[ 7.09635441 -3.21553559]
[ 7.1077678  -3.21168172]
[ 7.10584086 -3.21983414]
[ 7.10991707 -3.21845776]
[ 7.10922888 -3.22136934]
[ 7.11068467 -3.22087777]
[ 7.11043889 -3.22191762]
[ 7.11095881 -3.22174206]
[ 7.11087103 -3.22211344]
[ 7.11105672 -3.22205074]
[ 7.11102537 -3.22218337]
[ 7.11109168 -3.22216098]
[ 7.11108049 -3.22220835]
Accurate after 24 iterations


#### Metoda Gaussa-Seidela

Opiera się na tym samym pomyśle, co metoda Jacobiego, ale przy innym rozkładzie macierzy $A$:

$$
A=L_{*}+U\qquad {\text{where}}\qquad L_{*}={\begin{bmatrix}a_{11}&0&\cdots &0\\a_{21}&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}},\quad U={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\0&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &0\end{bmatrix}}.
$$

Wtedy układ równań możemy zapisać jako: $ L_{*}\mathbf {x} =\mathbf {b} -U\mathbf {x} $ i iterować tak:

$$ \mathbf {x} ^{(k+1)}=L_{*}^{-1}(\mathbf {b} -U\mathbf {x} ^{(k)}). $$

Element po elemencie:

$$ {\displaystyle x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum _{j=i+1}^{n}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\dots ,n.} $$

Podobnie jak z Jacobim, tutaj trick polega na tym, że macierz L jest łatwa do odwrócenia.

### Zadanie 2.
Zaimplementować metodę Gaussa-Seidela i przetestować na tych samych układach równań, co metodę Jacobiego. 

In [29]:
def gauss_seidel_solve(A: np.matrix, b: np.matrix, iterations = 1000) -> np.matrix:
    if A.shape[0] != A.shape[1] or A.shape[1] != b.shape[0]:
        raise ArithmeticError("a should be n * n matrix and b should be n * 1 matrix for some n > 0")
    n = A.shape[0]
    b1 = np.transpose(b)[0]
    x = np.zeros(n)
    k=0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0, i):
                sumL = sumL + A[i,j] * x1[j]
            for j in range(i+1, n):
                sumU = sumU + A[i,j] * x[j]
            x1[i] = (1 / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k = k + 1
    return x

In [30]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, gauss_seidel_solve)

[ 5.5        -2.07142857]
[ 6.53571429 -2.81122449]
[ 6.90561224 -3.07543732]
[ 7.03771866 -3.16979904]
[ 7.08489952 -3.20349966]
[ 7.10174983 -3.21553559]
[ 7.1077678  -3.21983414]
[ 7.10991707 -3.22136934]
[ 7.11068467 -3.22191762]
[ 7.11095881 -3.22211344]
[ 7.11105672 -3.22218337]
[ 7.11109168 -3.22220835]
Accurate after 12 iterations


#### Metoda SOR (Successive Over Relaxation)

Znowu podobnie, tyle, że tym razem rozkładamy macierz na sumę trzech macierzy:

$$
D={\begin{bmatrix}a_{11}&0&\cdots &0\\0&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &a_{nn}\end{bmatrix}},\quad L={\begin{bmatrix}0&0&\cdots &0\\a_{21}&0&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &0\end{bmatrix}},\quad U={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\0&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &0\end{bmatrix}}.
$$

Co umożliwia zapisanie układu równań tak: $ (D+\omega L)\mathbf {x} =\omega \mathbf {b} -[\omega U+(\omega -1)D]\mathbf {x} $ i daje następujące wzory na iterację:

$$ \mathbf {x} ^{(k+1)}=(D+\omega L)^{-1}{\big (}\omega \mathbf {b} -[\omega U+(\omega -1)D]\mathbf {x} ^{(k)}{\big )}=L_{w}\mathbf {x} ^{(k)}+\mathbf {c} , $$

oraz:

$$ x_{i}^{(k+1)}=(1-\omega )x_{i}^{(k)}+{\frac {\omega }{a_{ii}}}\left(b_{i}-\sum _{j<i}a_{ij}x_{j}^{(k+1)}-\sum _{j>i}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\ldots ,n. $$

### Zadanie 3.
Zaimplementować metodę SOR i przetestować na tych samych układach.

In [37]:
def sor_solve(A: np.matrix, b: np.matrix, iterations) -> np.matrix:
    if A.shape[0] != A.shape[1] or A.shape[1] != b.shape[0]:
        raise ArithmeticError("a should be n * n matrix and b should be n * 1 matrix for some n > 0")
    n = A.shape[0]
    b1 = np.transpose(b)[0]
    x = np.zeros(n)
    omega = 0.4
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0, i):
                sumL = sumL + A[i,j] * x1[j]
            for j in range(i+1, n):
                sumU = sumU + A[i,j] * x[j]
            x1[i] = (1 - omega) * x[i] + (omega / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k = k + 1
    return x

In [38]:
A = np.matrix([[2, 1], [5, 7]])
b = np.matrix([11, 13]).transpose()
check_method(A, b, sor_solve)

[2.2        0.11428571]
[ 3.49714286 -0.1877551 ]
[ 4.33583673 -0.60860641]
[ 4.92322332 -1.02894194]
[ 5.35972238 -1.40585727]
[ 5.69700488 -1.7283729 ]
[ 5.96387751 -1.9981316 ]
[ 6.17795283 -2.2211512 ]
[ 6.35100194 -2.40440556]
[ 6.49148227 -2.55449541]
[ 6.60578845 -2.67720823]
[ 6.69891471 -2.77744343]
[ 6.77483751 -2.85927678]
[ 6.83675786 -2.92606831]
[ 6.88726838 -2.98057481]
[ 6.92847599 -3.02505231]
[ 6.96209606 -3.06134455]
[ 6.98952654 -3.09095717]
[ 7.01190736 -3.11511926]
[ 7.03016827 -3.13483392]
[ 7.04506774 -3.15091971]
[ 7.05722459 -3.16404456]
[ 7.06714367 -3.1747535 ]
[ 7.0752369  -3.18349121]
[ 7.08184038 -3.19062055]
[ 7.08722834 -3.19643757]
[ 7.09162452 -3.20118383]
[ 7.09521148 -3.20505644]
[ 7.09813817 -3.2082162 ]
[ 7.10052614 -3.21079433]
[ 7.10247455 -3.2128979 ]
[ 7.10406431 -3.21461426]
[ 7.10536144 -3.21601468]
[ 7.1064198  -3.21715732]
[ 7.10728334 -3.21808963]
[ 7.10798793 -3.21885033]
[ 7.10856283 -3.21947101]
[ 7.1090319  -3.21997743]
[ 7.10941462 -

### Zadanie 4.
Dla powyższych metod porównać (na wykresie) tempo zbiegania do rozwiązania.

### Pytanie
Jakie jest kryterium zbieżności metod powyżej? Czy zawsze można je stosować?

#### Bonus:
Jak przeklejać piękne wzory z Wikipedii i się przy tym nie namęczyć? (na zajęciach).